In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext 
spark = SparkSession.builder.appName("Python Spark SQL Hive integration example").enableHiveSupport().getOrCreate()
sc = spark.sparkContext
spark

In [76]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  student|      false|
| default|   xxm_t1|      false|
| default|   xxm_t2|      false|
| default|   xxm_t3|      false|
| default|   xxm_t4|      false|
| default|   xxm_t5|      false|
|        |temptable|       true|
|        |     tmpv|       true|
+--------+---------+-----------+



In [77]:
sql="""
SELECT cookieid,
createtime,
pv,
SUM(pv) OVER(PARTITION BY cookieid ORDER BY createtime) AS pv1, -- 默认为从起点到当前行
SUM(pv) OVER(PARTITION BY cookieid ORDER BY createtime ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS pv2, --从起点到当前行，结果同pv1
SUM(pv) OVER(PARTITION BY cookieid) AS pv3,                --分组内所有行
SUM(pv) OVER(PARTITION BY cookieid ORDER BY createtime ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS pv4,   --当前行+往前3行
SUM(pv) OVER(PARTITION BY cookieid ORDER BY createtime ROWS BETWEEN 3 PRECEDING AND 1 FOLLOWING) AS pv5,    --当前行+往前3行+往后1行
SUM(pv) OVER(PARTITION BY cookieid ORDER BY createtime ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS pv6   ---当前行+往后所有行  
FROM xxm_t1
"""
spark.sql(sql).show()

+--------+----------+---+---+---+---+---+---+---+
|cookieid|createtime| pv|pv1|pv2|pv3|pv4|pv5|pv6|
+--------+----------+---+---+---+---+---+---+---+
| cookie1|2015-04-10|  1|  2|  1| 52|  1|  2| 52|
| cookie1|2015-04-10|  1|  2|  2| 52|  2|  7| 51|
| cookie1|2015-04-11|  5| 12|  7| 52|  7| 12| 50|
| cookie1|2015-04-11|  5| 12| 12| 52| 12| 19| 45|
| cookie1|2015-04-12|  7| 26| 19| 52| 18| 25| 40|
| cookie1|2015-04-12|  7| 26| 26| 52| 24| 27| 33|
| cookie1|2015-04-13|  3| 32| 29| 52| 22| 25| 26|
| cookie1|2015-04-13|  3| 32| 32| 52| 20| 22| 23|
| cookie1|2015-04-14|  2| 36| 34| 52| 15| 17| 20|
| cookie1|2015-04-14|  2| 36| 36| 52| 10| 14| 18|
| cookie1|2015-04-15|  4| 44| 40| 52| 11| 15| 16|
| cookie1|2015-04-15|  4| 44| 44| 52| 12| 16| 12|
| cookie1|2015-04-16|  4| 52| 48| 52| 14| 18|  8|
| cookie1|2015-04-16|  4| 52| 52| 52| 16| 16|  4|
+--------+----------+---+---+---+---+---+---+---+

